Wichtig:
- Freitag:
- iso-country code 3-stellig anstelle von dem zweistelligen (aus der gleichen Tabelle)
  bei Apple,Google,Flights und Corona
- Google Datensatz amputieren (am 1.01.21) beginnen lassen und am Ende hinzufügen
- Google nur bis 2020
- Flights nur 2020
- Corona Daten 1.01 und 2.01 amputieren
- Länder mit Überschneidungen finden und Übrige löschen

- Schema für die Tabellen in Postgresql einrichten, (Iso-code,date) als Composite Key
- Exportieren

Vlt:
- Multipolygons der Länder in Größe reduzieren 
- Dates einschränken(z.B. bis Ende 2020) 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import datetime

# Apple Dataset

In [27]:
df_apple = pd.read_csv(r'GeospacialDBs_Data/applemobilitytrends-2021-01-22.csv', 
                       header=0,  dtype={"alternative_name": str})
df_apple.head(5)

,geo_type,region,transportation_type,alternative_name,sub-region,country,2020-01-13,2020-01-14,2020-01-15,2020-01-16,...,2021-01-13,2021-01-14,2021-01-15,2021-01-16,2021-01-17,2021-01-18,2021-01-19,2021-01-20,2021-01-21,2021-01-22
0,country/region,Albania,driving,NaN,NaN,NaN,100.0,95.30,101.43,97.20,...,112.99,112.01,125.27,145.95,134.65,110.75,110.08,114.76,108.25,122.22
1,country/region,Albania,walking,NaN,NaN,NaN,100.0,100.68,98.93,98.46,...,138.08,146.48,155.22,169.13,122.17,130.02,134.37,131.90,130.58,144.09
2,country/region,Argentina,driving,NaN,NaN,NaN,100.0,97.07,102.45,111.21,...,84.53,90.94,100.34,103.96,84.92,86.28,86.42,88.94,91.46,107.22
3,country/region,Argentina,walking,NaN,NaN,NaN,100.0,95.11,101.37,112.67,...,69.09,74.80,75.03,74.34,63.64,77.47,77.23,77.58,78.35,86.39
4,country/region,Australia,driving,AU,NaN,NaN,100.0,102.98,104.21,108.63,...,110.80,114.94,113.08,99.11,107.56,107.83,112.30,114.35,124.10,121.95


In [28]:
df_apple["geo_type"].unique()

array(['country/region', 'city', 'sub-region', 'county'], dtype=object)

In [29]:
df_apple_countries = df_apple[df_apple["geo_type"] == 'country/region'].copy()
df_apple_countries.drop(["geo_type","sub-region","country","alternative_name"], inplace=True, axis=1)
df_apple_countries

,region,transportation_type,2020-01-13,2020-01-14,2020-01-15,2020-01-16,2020-01-17,2020-01-18,2020-01-19,2020-01-20,...,2021-01-13,2021-01-14,2021-01-15,2021-01-16,2021-01-17,2021-01-18,2021-01-19,2021-01-20,2021-01-21,2021-01-22
0,Albania,driving,100.0,95.30,101.43,97.20,103.55,112.67,104.83,94.39,...,112.99,112.01,125.27,145.95,134.65,110.75,110.08,114.76,108.25,122.22
1,Albania,walking,100.0,100.68,98.93,98.46,100.85,100.13,82.13,95.65,...,138.08,146.48,155.22,169.13,122.17,130.02,134.37,131.90,130.58,144.09
2,Argentina,driving,100.0,97.07,102.45,111.21,118.45,124.01,95.44,95.13,...,84.53,90.94,100.34,103.96,84.92,86.28,86.42,88.94,91.46,107.22
3,Argentina,walking,100.0,95.11,101.37,112.67,116.72,114.14,84.54,101.37,...,69.09,74.80,75.03,74.34,63.64,77.47,77.23,77.58,78.35,86.39
4,Australia,driving,100.0,102.98,104.21,108.63,109.08,89.00,99.35,103.53,...,110.80,114.94,113.08,99.11,107.56,107.83,112.30,114.35,124.10,121.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,United States,walking,100.0,99.59,101.87,100.60,121.36,132.38,100.10,94.79,...,95.61,98.48,116.29,122.71,94.35,97.72,93.18,92.27,96.00,114.38
149,Uruguay,driving,100.0,96.84,92.90,103.48,114.91,115.03,99.48,93.30,...,75.83,76.73,89.39,81.79,69.43,78.07,74.86,74.62,77.43,89.53
150,Uruguay,walking,100.0,94.90,78.04,100.46,105.04,98.78,76.52,89.94,...,70.80,72.56,70.88,68.09,51.33,70.17,66.89,68.76,69.85,77.77
151,Vietnam,driving,100.0,104.96,108.67,107.09,121.07,130.40,123.99,122.77,...,89.33,93.08,112.49,121.86,99.37,88.86,89.84,93.29,94.23,109.57


In [30]:
df_list = []
for region in df_apple_countries["region"].unique():
    for transportation_type in df_apple_countries[df_apple_countries["region"] == region]["transportation_type"].unique():
        series_sub = df_apple_countries[(df_apple_countries["region"] == region) & (df_apple_countries["transportation_type"] == transportation_type)].iloc[:, 2:].transpose()
        df_list += [{'date': timestamp, 'country_region': region, 'transportation_type': transportation_type, 'percent_change_from_baseline': value[0]} for timestamp, value in zip(series_sub.index, series_sub.values)]
df_apple_reshaped = pd.DataFrame(df_list)

df_apple_reshaped

,date,country_region,transportation_type,percent_change_from_baseline
0,2020-01-13,Albania,driving,100.00
1,2020-01-14,Albania,driving,95.30
2,2020-01-15,Albania,driving,101.43
3,2020-01-16,Albania,driving,97.20
4,2020-01-17,Albania,driving,103.55
...,...,...,...,...
57523,2021-01-18,Vietnam,walking,119.00
57524,2021-01-19,Vietnam,walking,120.13
57525,2021-01-20,Vietnam,walking,124.09
57526,2021-01-21,Vietnam,walking,127.39


In [31]:
for i in range(len(df_apple_reshaped)):
    df_apple_reshaped.at[i,'percent_change_from_baseline']-=100

df_apple_reshaped

,date,country_region,transportation_type,percent_change_from_baseline
0,2020-01-13,Albania,driving,0.00
1,2020-01-14,Albania,driving,-4.70
2,2020-01-15,Albania,driving,1.43
3,2020-01-16,Albania,driving,-2.80
4,2020-01-17,Albania,driving,3.55
...,...,...,...,...
57523,2021-01-18,Vietnam,walking,19.00
57524,2021-01-19,Vietnam,walking,20.13
57525,2021-01-20,Vietnam,walking,24.09
57526,2021-01-21,Vietnam,walking,27.39


## Add iso-country codes

In [32]:
df_iso_codes = pd.read_csv(r'GeospacialDBs_Data/wikipedia-iso-country-codes.csv', header=0, sep=",")
df_iso_codes

,English short name lower case,Alpha-2 code,Alpha-3 code,Numeric code,ISO 3166-2
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX
2,Albania,AL,ALB,8,ISO 3166-2:AL
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ
4,American Samoa,AS,ASM,16,ISO 3166-2:AS
...,...,...,...,...,...
241,Wallis and Futuna,WF,WLF,876,ISO 3166-2:WF
242,Western Sahara,EH,ESH,732,ISO 3166-2:EH
243,Yemen,YE,YEM,887,ISO 3166-2:YE
244,Zambia,ZM,ZMB,894,ISO 3166-2:ZM


In [33]:
df_iso_codes.set_index('English short name lower case',inplace=True)
df_iso_codes

,Alpha-2 code,Alpha-3 code,Numeric code,ISO 3166-2
English short name lower case,,,,
Afghanistan,AF,AFG,4,ISO 3166-2:AF
Åland Islands,AX,ALA,248,ISO 3166-2:AX
Albania,AL,ALB,8,ISO 3166-2:AL
Algeria,DZ,DZA,12,ISO 3166-2:DZ
American Samoa,AS,ASM,16,ISO 3166-2:AS
...,...,...,...,...
Wallis and Futuna,WF,WLF,876,ISO 3166-2:WF
Western Sahara,EH,ESH,732,ISO 3166-2:EH
Yemen,YE,YEM,887,ISO 3166-2:YE


In [34]:
countries={}
for i in range(len(df_apple_reshaped)):
    tmp=df_apple_reshaped['country_region'][i]
    if tmp not in df_iso_codes.index:
        if tmp not in countries:
            countries[tmp]=1

countries  

{'Republic of Korea': 1, 'Russia': 1, 'Taiwan': 1, 'Vietnam': 1}

In [35]:
for i in df_iso_codes.index:
    print(i)

Afghanistan
Åland Islands
Albania
Algeria
American Samoa
Andorra
Angola
Anguilla
Antarctica
Antigua and Barbuda
Argentina
Armenia
Aruba
Australia
Austria
Azerbaijan
Bahamas
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bermuda
Bhutan
Bolivia, Plurinational State of
Bosnia and Herzegovina
Botswana
Bouvet Island
Brazil
British Indian Ocean Territory
Brunei Darussalam
Bulgaria
Burkina Faso
Burundi
Cambodia
Cameroon
Canada
Cape Verde
Cayman Islands
Central African Republic
Chad
Chile
China
Christmas Island
Cocos (Keeling) Islands
Colombia
Comoros
Congo
Congo, the Democratic Republic of the
Cook Islands
Costa Rica
Côte d'Ivoire
Croatia
Cuba
Cyprus
Czech Republic
Denmark
Djibouti
Dominica
Dominican Republic
Ecuador
Egypt
El Salvador
Equatorial Guinea
Eritrea
Estonia
Ethiopia
Falkland Islands (Malvinas)
Faroe Islands
Fiji
Finland
France
French Guiana
French Polynesia
French Southern Territories
Gabon
Gambia
Georgia
Germany
Ghana
Gibraltar
Greece
Greenland
Grenada
Guadeloupe
Guam
Gu

In [36]:
#translate the missing ones
countries['Republic of Korea']="Korea, Republic of"
countries['Russia']='Russian Federation'
countries['Taiwan']='Taiwan, Province of China'
countries['Vietnam']='Viet Nam'

countries


{'Republic of Korea': 'Korea, Republic of',
 'Russia': 'Russian Federation',
 'Taiwan': 'Taiwan, Province of China',
 'Vietnam': 'Viet Nam'}

In [37]:
df_apple_reshaped['country_region_code']='Placeholder'
df_apple_reshaped

,date,country_region,transportation_type,percent_change_from_baseline,country_region_code
0,2020-01-13,Albania,driving,0.00,Placeholder
1,2020-01-14,Albania,driving,-4.70,Placeholder
2,2020-01-15,Albania,driving,1.43,Placeholder
3,2020-01-16,Albania,driving,-2.80,Placeholder
4,2020-01-17,Albania,driving,3.55,Placeholder
...,...,...,...,...,...
57523,2021-01-18,Vietnam,walking,19.00,Placeholder
57524,2021-01-19,Vietnam,walking,20.13,Placeholder
57525,2021-01-20,Vietnam,walking,24.09,Placeholder
57526,2021-01-21,Vietnam,walking,27.39,Placeholder


In [38]:
for i in range(len(df_apple_reshaped)):
    tmp=df_apple_reshaped['country_region'][i]
    if tmp not in set(df_iso_codes.index):
        tmp=countries[tmp]
    df_apple_reshaped['country_region_code'][i]=df_iso_codes['Alpha-3 code'][tmp]

df_apple_reshaped
    
        

<ipython-input-38-2075bb7d37db>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_apple_reshaped['country_region_code'][i]=df_iso_codes['Alpha-3 code'][tmp]


,date,country_region,transportation_type,percent_change_from_baseline,country_region_code
0,2020-01-13,Albania,driving,0.00,ALB
1,2020-01-14,Albania,driving,-4.70,ALB
2,2020-01-15,Albania,driving,1.43,ALB
3,2020-01-16,Albania,driving,-2.80,ALB
4,2020-01-17,Albania,driving,3.55,ALB
...,...,...,...,...,...
57523,2021-01-18,Vietnam,walking,19.00,VNM
57524,2021-01-19,Vietnam,walking,20.13,VNM
57525,2021-01-20,Vietnam,walking,24.09,VNM
57526,2021-01-21,Vietnam,walking,27.39,VNM


In [39]:
df_apple_reshaped.drop(columns=['country_region'],inplace=True)
df_apple_reshaped

,date,transportation_type,percent_change_from_baseline,country_region_code
0,2020-01-13,driving,0.00,ALB
1,2020-01-14,driving,-4.70,ALB
2,2020-01-15,driving,1.43,ALB
3,2020-01-16,driving,-2.80,ALB
4,2020-01-17,driving,3.55,ALB
...,...,...,...,...
57523,2021-01-18,walking,19.00,VNM
57524,2021-01-19,walking,20.13,VNM
57525,2021-01-20,walking,24.09,VNM
57526,2021-01-21,walking,27.39,VNM


In [40]:
df_apple_reshaped['transportation_type'].unique()


array(['driving', 'walking', 'transit'], dtype=object)

In [41]:
print(df_apple_reshaped['date'].min())
print(df_apple_reshaped['date'].max())

2020-01-13
2021-01-22


In [42]:
#create new dataframe
df_apple_new=pd.DataFrame(columns=['date','country_region_code','driving_percent_change_from_baseline','walking_percent_change_from_baseline','transit_percent_change_from_baseline'])
df_apple_new


,date,country_region_code,driving_percent_change_from_baseline,walking_percent_change_from_baseline,transit_percent_change_from_baseline


In [44]:
#take all dates from 2020
numdays=366

base = datetime.date(2020,1,1)
date_list = [base + datetime.timedelta(days=x) for x in range(numdays)]
date_list
#pd.to_datetime(date_list[0])

for code in apple_codes:
    for date in date_list:
        if date==datetime.date(2020,1,1):
            df_apple_new.loc[str(date)+str(code)]=[date,code,0,0,0]
        else:
            df_apple_new.loc[str(date)+str(code)]=[date,code,np.nan,np.nan,np.nan]
        
df_apple_new

,date,country_region_code,driving_percent_change_from_baseline,walking_percent_change_from_baseline,transit_percent_change_from_baseline
2020-01-01ALB,2020-01-01,ALB,0,0,0
2020-01-02ALB,2020-01-02,ALB,NaN,NaN,NaN
2020-01-03ALB,2020-01-03,ALB,NaN,NaN,NaN
2020-01-04ALB,2020-01-04,ALB,NaN,NaN,NaN
2020-01-05ALB,2020-01-05,ALB,NaN,NaN,NaN
...,...,...,...,...,...
2020-12-27VNM,2020-12-27,VNM,NaN,NaN,NaN
2020-12-28VNM,2020-12-28,VNM,NaN,NaN,NaN
2020-12-29VNM,2020-12-29,VNM,NaN,NaN,NaN
2020-12-30VNM,2020-12-30,VNM,NaN,NaN,NaN


In [45]:
#copy the values to new table
for row in df_apple_reshaped.iterrows():
    key=row[1]['date']+row[1]['country_region_code']
    if key in df_apple_new.index:
        if row[1]['transportation_type']=='driving':
            df_apple_new.loc[key]['driving_percent_change_from_baseline']=row[1]['percent_change_from_baseline']
        elif row[1]['transportation_type']=='walking':
            df_apple_new.loc[key]['walking_percent_change_from_baseline']=row[1]['percent_change_from_baseline']
        else:
            df_apple_new.loc[key]['transit_percent_change_from_baseline']=row[1]['percent_change_from_baseline']
df_apple_new

,date,country_region_code,driving_percent_change_from_baseline,walking_percent_change_from_baseline,transit_percent_change_from_baseline
2020-01-01ALB,2020-01-01,ALB,0,0,0
2020-01-02ALB,2020-01-02,ALB,NaN,NaN,NaN
2020-01-03ALB,2020-01-03,ALB,NaN,NaN,NaN
2020-01-04ALB,2020-01-04,ALB,NaN,NaN,NaN
2020-01-05ALB,2020-01-05,ALB,NaN,NaN,NaN
...,...,...,...,...,...
2020-12-27VNM,2020-12-27,VNM,-2.07,29.77,NaN
2020-12-28VNM,2020-12-28,VNM,-6.06,21.47,NaN
2020-12-29VNM,2020-12-29,VNM,-7.53,22.4,NaN
2020-12-30VNM,2020-12-30,VNM,0.86,29.61,NaN


In [46]:
df_apple_new.isnull().sum()

date                                        0
country_region_code                         0
driving_percent_change_from_baseline      819
walking_percent_change_from_baseline      819
transit_percent_change_from_baseline    13491
dtype: int64

In [47]:
#change dtypes, so that interpolation can be used
convert_dict={
    'driving_percent_change_from_baseline': float,
    'walking_percent_change_from_baseline': float,
    'transit_percent_change_from_baseline': float
}

df_apple_new = df_apple_new.astype(convert_dict)

df_apple_new.dtypes

date                                     object
country_region_code                      object
driving_percent_change_from_baseline    float64
walking_percent_change_from_baseline    float64
transit_percent_change_from_baseline    float64
dtype: object

In [48]:
df_apple_new.index=pd.RangeIndex(len(df_apple_new.index))
df_apple_new

,date,country_region_code,driving_percent_change_from_baseline,walking_percent_change_from_baseline,transit_percent_change_from_baseline
0,2020-01-01,ALB,0.00,0.00,0.0
1,2020-01-02,ALB,NaN,NaN,NaN
2,2020-01-03,ALB,NaN,NaN,NaN
3,2020-01-04,ALB,NaN,NaN,NaN
4,2020-01-05,ALB,NaN,NaN,NaN
...,...,...,...,...,...
23053,2020-12-27,VNM,-2.07,29.77,NaN
23054,2020-12-28,VNM,-6.06,21.47,NaN
23055,2020-12-29,VNM,-7.53,22.40,NaN
23056,2020-12-30,VNM,0.86,29.61,NaN


In [29]:
#interpolation
percent_change_driving=df_apple_new['driving_percent_change_from_baseline'].interpolate(method='quadratic')
percent_change_walking=df_apple_new['walking_percent_change_from_baseline'].interpolate(method='quadratic')



df_apple_new['driving_percent_change_from_baseline']=percent_change_driving
df_apple_new['walking_percent_change_from_baseline']=percent_change_walking
df_apple_new.drop(columns=['transit_percent_change_from_baseline'],inplace=True)


df_apple_new

,date,country_region_code,driving_percent_change_from_baseline,walking_percent_change_from_baseline
0,2020-01-01,ALB,0.000000,0.000000
1,2020-01-02,ALB,5.983188,-0.970299
2,2020-01-03,ALB,10.878524,-1.764180
3,2020-01-04,ALB,14.686008,-2.381644
4,2020-01-05,ALB,17.405639,-2.822689
...,...,...,...,...
23053,2020-12-27,VNM,-2.070000,29.770000
23054,2020-12-28,VNM,-6.060000,21.470000
23055,2020-12-29,VNM,-7.530000,22.400000
23056,2020-12-30,VNM,0.860000,29.610000


In [30]:
df_apple_new.isnull().sum()

date                                    0
country_region_code                     0
driving_percent_change_from_baseline    0
walking_percent_change_from_baseline    0
dtype: int64

In [31]:
df_apple_new.to_csv(r"GeospacialDBs_Data/AppleMobilityData_reshaped.csv", index = False)

# Google Dataset

In [103]:
df_google = pd.read_csv('GeospacialDBs_Data/Global_Mobility_Report.csv', header=0,  dtype={"metro_area": str})
df_google

/home/julian/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,AE,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3952072,ZW,Zimbabwe,Midlands Province,Kwekwe,NaN,NaN,NaN,2021-01-13,NaN,NaN,NaN,NaN,-35.0,NaN
3952073,ZW,Zimbabwe,Midlands Province,Kwekwe,NaN,NaN,NaN,2021-01-14,NaN,NaN,NaN,NaN,-40.0,NaN
3952074,ZW,Zimbabwe,Midlands Province,Kwekwe,NaN,NaN,NaN,2021-01-15,NaN,NaN,NaN,NaN,-41.0,NaN
3952075,ZW,Zimbabwe,Midlands Province,Kwekwe,NaN,NaN,NaN,2021-01-18,NaN,NaN,NaN,NaN,-41.0,NaN


In [104]:
df_google.drop(["sub_region_1","sub_region_2","metro_area","iso_3166_2_code","census_fips_code"], inplace=True, axis=1)
df_google.head(5)

,country_region_code,country_region,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AE,United Arab Emirates,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,United Arab Emirates,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,AE,United Arab Emirates,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,AE,United Arab Emirates,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,AE,United Arab Emirates,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0


In [105]:
df_iso_codes = pd.read_csv(r'GeospacialDBs_Data/wikipedia-iso-country-codes.csv', header=0, sep=",")
df_iso_codes.set_index(['Alpha-2 code'],inplace=True)
df_iso_codes

,English short name lower case,Alpha-3 code,Numeric code,ISO 3166-2
Alpha-2 code,,,,
AF,Afghanistan,AFG,4,ISO 3166-2:AF
AX,Åland Islands,ALA,248,ISO 3166-2:AX
AL,Albania,ALB,8,ISO 3166-2:AL
DZ,Algeria,DZA,12,ISO 3166-2:DZ
AS,American Samoa,ASM,16,ISO 3166-2:AS
...,...,...,...,...
WF,Wallis and Futuna,WLF,876,ISO 3166-2:WF
EH,Western Sahara,ESH,732,ISO 3166-2:EH
YE,Yemen,YEM,887,ISO 3166-2:YE


In [ ]:
for i in range(len(df_google)):
    country_code=df_google.at[i,'country_region_code']
    df_google.at[i,'country_region_code']=df_iso_codes.at[country_code,'Alpha-3 code']

df_google 

In [ ]:
df_google.drop(columns=['country_region'],inplace=True)
df_google

In [ ]:
df_google

In [11]:
df_google.to_csv(r"GeospacialDBs_Data/Global_Mobility_Report_reshaped.csv", index = False)

# Flight Data

In [47]:
df_airport_codes = pd.read_csv(r'GeospacialDBs_Data/airport-codes.csv' 
, header=0, sep=",")
df_airport_codes.head(5)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"40.07080078125, -74.93360137939453"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"38.704022, -101.473911"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"59.94919968, -151.695999146"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"34.86479949951172, -86.77030181884766"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"35.6087, -91.254898"


In [48]:
df_airport_codes[df_airport_codes['iso_country'].isnull()]

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
21432,FYAA,small_airport,Ai-Ais Airport,2000.0,AF,NaN,NA-KU,Ai-Ais,FYAA,AIW,NaN,"-27.995, 17.5966"
21433,FYAB,small_airport,Aroab B Airport,3235.0,AF,NaN,NA-KA,Aroab,FYAB,NaN,NaN,"-26.776100158691406, 19.633100509643555"
21434,FYAK,small_airport,Aussenkehr Airport,970.0,AF,NaN,NA-KA,Aussenkehr,FYAK,NaN,NaN,"-28.4587, 17.4645"
21435,FYAM,small_airport,Aminuis Airstrip,4012.0,AF,NaN,NA-OH,Aminuis,FYAM,NaN,NaN,"-23.655799865722656, 19.351699829101562"
21436,FYAR,medium_airport,Arandis Airport,1905.0,AF,NaN,NA-ER,Arandis,FYAR,ADI,NaN,"-22.462200164794922, 14.979999542236328"
...,...,...,...,...,...,...,...,...,...,...,...,...
35695,NA-0172,small_airport,Haklesdoorn Airport,NaN,AF,NaN,NA-KA,Haklesdoorn,NaN,NaN,NaN,"-28.886092, 18.239166"
35696,NA-0173,small_airport,Ndonga Linena Airstrip,NaN,AF,NaN,NA-KW,Shitemo,NaN,NaN,NaN,"-17.98345, 20.534708"
35697,NA-0174,small_airport,Farm Etemba Landing Strip,3503.0,AF,NaN,NA-ER,Etemba Wilderness Camp,NaN,NaN,NaN,"-21.405111, 15.606778"
35698,NA-0175,small_airport,Farm Dawib Ost Landing Strip,3503.0,AF,NaN,NA-ER,Hohenstein Lodge,NaN,NaN,NaN,"-21.774528, 15.515361"


In [110]:
first_flag = True
for filename in glob.glob(r'GeospacialDBs_Data/flightlist_*.csv'):
    df_flight = pd.read_csv(filename, header=0, sep=",",  dtype={"number": str})
    country_map = {ident:iso_country for ident, iso_country in zip(df_airport_codes["ident"],df_airport_codes["iso_country"])}
    daily_grouped = df_flight.groupby(['origin', "day"])["callsign"].agg(['count'])
    daily_index = daily_grouped.index.get_level_values(0)
    uniques = df_airport_codes["ident"].unique()
    daily_grouped['iso_country'] = daily_grouped.index.get_level_values(0).map(country_map)
    country_daily_origin = daily_grouped.groupby(['iso_country', "day"])['count'].agg(['count'])
    daily_grouped2 = df_flight.groupby(['destination', "day"])["callsign"].agg(['count'])
    daily_index2 = daily_grouped2.index.get_level_values(0)
    uniques = df_airport_codes["ident"].unique()
    daily_grouped2['iso_country'] = daily_grouped2.index.get_level_values(0).map(country_map)
    country_daily_destination = daily_grouped2.groupby(['iso_country', "day"])["count"].agg(['count'])
    country_daily = pd.concat([country_daily_origin, country_daily_destination], axis=1)
    country_daily.columns = ['origin_count', 'destination_count']
    if first_flag:
        country_flights_io = country_daily
        first_flag = False
    else:
        country_flights_io = country_flights_io.append([country_daily])
country_flights_io.head(10)

origin_count  destination_count
iso_country day                                                       
AE          2019-04-01 00:00:00+00:00           7.0                8.0
            2019-04-02 00:00:00+00:00           7.0                7.0
            2019-04-03 00:00:00+00:00           6.0               13.0
            2019-04-04 00:00:00+00:00          11.0               12.0
            2019-04-05 00:00:00+00:00           5.0                7.0
            2019-04-06 00:00:00+00:00           6.0                8.0
            2019-04-07 00:00:00+00:00           6.0                7.0
            2019-04-08 00:00:00+00:00           6.0                9.0
            2019-04-09 00:00:00+00:00           7.0                9.0
            2019-04-10 00:00:00+00:00           5.0               10.0

In [111]:
country_flights_io.reset_index(inplace=True)
country_flights_io

,iso_country,day,origin_count,destination_count
0,AE,2019-04-01 00:00:00+00:00,7.0,8.0
1,AE,2019-04-02 00:00:00+00:00,7.0,7.0
2,AE,2019-04-03 00:00:00+00:00,6.0,13.0
3,AE,2019-04-04 00:00:00+00:00,11.0,12.0
4,AE,2019-04-05 00:00:00+00:00,5.0,7.0
...,...,...,...,...
65667,ZA,2019-06-26 00:00:00+00:00,16.0,17.0
65668,ZA,2019-06-27 00:00:00+00:00,12.0,14.0
65669,ZA,2019-06-28 00:00:00+00:00,14.0,16.0
65670,ZA,2019-06-29 00:00:00+00:00,17.0,24.0


In [114]:
df_iso_codes = pd.read_csv(r'GeospacialDBs_Data/wikipedia-iso-country-codes.csv', header=0, sep=",")
df_iso_codes.set_index('Alpha-2 code',inplace=True)
df_iso_codes.head(5)


,English short name lower case,Alpha-3 code,Numeric code,ISO 3166-2
Alpha-2 code,,,,
AF,Afghanistan,AFG,4,ISO 3166-2:AF
AX,Åland Islands,ALA,248,ISO 3166-2:AX
AL,Albania,ALB,8,ISO 3166-2:AL
DZ,Algeria,DZA,12,ISO 3166-2:DZ
AS,American Samoa,ASM,16,ISO 3166-2:AS


In [115]:
country_flights_io.rename(columns={'iso_country':'country_region_code','day':'date'},inplace=True)
country_flights_io

,country_region_code,date,origin_count,destination_count
0,AE,2019-04-01 00:00:00+00:00,7.0,8.0
1,AE,2019-04-02 00:00:00+00:00,7.0,7.0
2,AE,2019-04-03 00:00:00+00:00,6.0,13.0
3,AE,2019-04-04 00:00:00+00:00,11.0,12.0
4,AE,2019-04-05 00:00:00+00:00,5.0,7.0
...,...,...,...,...
65667,ZA,2019-06-26 00:00:00+00:00,16.0,17.0
65668,ZA,2019-06-27 00:00:00+00:00,12.0,14.0
65669,ZA,2019-06-28 00:00:00+00:00,14.0,16.0
65670,ZA,2019-06-29 00:00:00+00:00,17.0,24.0


In [116]:
for i in range(len(country_flights_io)):
    country_code=country_flights_io.at[i,'country_region_code']
    if country_code in df_iso_codes.index:
        country_flights_io.at[i,'country_region_code']=df_iso_codes.at[country_code,'Alpha-3 code']

country_flights_io

,country_region_code,date,origin_count,destination_count
0,ARE,2019-04-01 00:00:00+00:00,7.0,8.0
1,ARE,2019-04-02 00:00:00+00:00,7.0,7.0
2,ARE,2019-04-03 00:00:00+00:00,6.0,13.0
3,ARE,2019-04-04 00:00:00+00:00,11.0,12.0
4,ARE,2019-04-05 00:00:00+00:00,5.0,7.0
...,...,...,...,...
65667,ZAF,2019-06-26 00:00:00+00:00,16.0,17.0
65668,ZAF,2019-06-27 00:00:00+00:00,12.0,14.0
65669,ZAF,2019-06-28 00:00:00+00:00,14.0,16.0
65670,ZAF,2019-06-29 00:00:00+00:00,17.0,24.0


In [119]:
country_flights_io.isnull().sum()

country_region_code       0
date                      0
origin_count           3504
destination_count       711
dtype: int64

In [120]:
#replace NaNs with Zeros
country_flights_io.fillna(0,inplace=True)

In [121]:
country_flights_io.isnull().sum()

country_region_code    0
date                   0
origin_count           0
destination_count      0
dtype: int64

In [49]:
country_flights_io=pd.read_csv('GeospacialDBs_Data/Flights_io_per_day_and_country.csv',header=0)

In [180]:
country_flights_io['date']=pd.to_datetime(country_flights_io['date'])
country_flights_io['date']=country_flights_io['date'].dt.date

In [51]:
country_flights_io

,country_region_code,date,origin_count,destination_count
0,ARE,2019-04-01,7.0,8.0
1,ARE,2019-04-02,7.0,7.0
2,ARE,2019-04-03,6.0,13.0
3,ARE,2019-04-04,11.0,12.0
4,ARE,2019-04-05,5.0,7.0
...,...,...,...,...
65667,ZAF,2019-06-26,16.0,17.0
65668,ZAF,2019-06-27,12.0,14.0
65669,ZAF,2019-06-28,14.0,16.0
65670,ZAF,2019-06-29,17.0,24.0


In [182]:
country_flights_io.to_csv(r'GeospacialDBs_Data/Flights_io_per_day_and_country.csv', index = False)

# Corona Data

In [89]:
df_cases=pd.read_csv(r"GeospacialDBs_Data/WHO-COVID-19-global-data.csv",header=0)
df_cases

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-03,AF,Afghanistan,EMRO,0,0,0,0
1,2020-01-04,AF,Afghanistan,EMRO,0,0,0,0
2,2020-01-05,AF,Afghanistan,EMRO,0,0,0,0
3,2020-01-06,AF,Afghanistan,EMRO,0,0,0,0
4,2020-01-07,AF,Afghanistan,EMRO,0,0,0,0
...,...,...,...,...,...,...,...,...
94209,2021-01-29,ZW,Zimbabwe,AFRO,342,32646,38,1160
94210,2021-01-30,ZW,Zimbabwe,AFRO,306,32952,18,1178
94211,2021-01-31,ZW,Zimbabwe,AFRO,319,33271,15,1193
94212,2021-02-01,ZW,Zimbabwe,AFRO,117,33388,24,1217


In [90]:
df_cases.drop(columns=['WHO_region','Country'],inplace=True)
df_cases

,Date_reported,Country_code,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-03,AF,0,0,0,0
1,2020-01-04,AF,0,0,0,0
2,2020-01-05,AF,0,0,0,0
3,2020-01-06,AF,0,0,0,0
4,2020-01-07,AF,0,0,0,0
...,...,...,...,...,...,...
94209,2021-01-29,ZW,342,32646,38,1160
94210,2021-01-30,ZW,306,32952,18,1178
94211,2021-01-31,ZW,319,33271,15,1193
94212,2021-02-01,ZW,117,33388,24,1217


In [91]:
df_cases.columns=['date','country_region_code', 'new_cases','cumulative_cases','new_deaths','cumulative_deaths']
df_cases

,date,country_region_code,new_cases,cumulative_cases,new_deaths,cumulative_deaths
0,2020-01-03,AF,0,0,0,0
1,2020-01-04,AF,0,0,0,0
2,2020-01-05,AF,0,0,0,0
3,2020-01-06,AF,0,0,0,0
4,2020-01-07,AF,0,0,0,0
...,...,...,...,...,...,...
94209,2021-01-29,ZW,342,32646,38,1160
94210,2021-01-30,ZW,306,32952,18,1178
94211,2021-01-31,ZW,319,33271,15,1193
94212,2021-02-01,ZW,117,33388,24,1217


In [92]:
df_iso_codes = pd.read_csv(r'GeospacialDBs_Data/wikipedia-iso-country-codes.csv', header=0, sep=",")
df_iso_codes.set_index(['Alpha-2 code'],inplace=True)
df_iso_codes

,English short name lower case,Alpha-3 code,Numeric code,ISO 3166-2
Alpha-2 code,,,,
AF,Afghanistan,AFG,4,ISO 3166-2:AF
AX,Åland Islands,ALA,248,ISO 3166-2:AX
AL,Albania,ALB,8,ISO 3166-2:AL
DZ,Algeria,DZA,12,ISO 3166-2:DZ
AS,American Samoa,ASM,16,ISO 3166-2:AS
...,...,...,...,...
WF,Wallis and Futuna,WLF,876,ISO 3166-2:WF
EH,Western Sahara,ESH,732,ISO 3166-2:EH
YE,Yemen,YEM,887,ISO 3166-2:YE


In [93]:
for i in range(len(df_cases)):
    country_code=df_cases.at[i,'country_region_code']
    if country_code in df_iso_codes.index:
        df_cases.at[i,'country_region_code']=df_iso_codes.at[country_code,'Alpha-3 code']

df_cases

,date,country_region_code,new_cases,cumulative_cases,new_deaths,cumulative_deaths
0,2020-01-03,AFG,0,0,0,0
1,2020-01-04,AFG,0,0,0,0
2,2020-01-05,AFG,0,0,0,0
3,2020-01-06,AFG,0,0,0,0
4,2020-01-07,AFG,0,0,0,0
...,...,...,...,...,...,...
94209,2021-01-29,ZWE,342,32646,38,1160
94210,2021-01-30,ZWE,306,32952,18,1178
94211,2021-01-31,ZWE,319,33271,15,1193
94212,2021-02-01,ZWE,117,33388,24,1217


In [19]:
df_slices=[]
tmp_code=df_cases['country_region_code'][0]
tmp_num=0
for i in range(len(df_cases)):
    if df_cases['country_region_code'][i]!=tmp_code:
        df_slices.append(df_cases.iloc[tmp_num:i])
        tmp_num=i
    tmp_code=df_cases['country_region_code'][i]
df_slices.append(df_cases.iloc[tmp_num:len(df_cases)-1])

for j in range(len(df_slices)):
    start_data=[]
    start_data.insert(0,{'date': '2020-01-02','country_region_code': df_slices[j].iloc[0]['country_region_code'],'new_cases': 0, 'cumulative_cases': 0, 'new_deaths':0,'cumulative_deaths':0})
    start_data.insert(0,{'date': '2020-01-01','country_region_code': df_slices[j].iloc[0]['country_region_code'],'new_cases': 0, 'cumulative_cases': 0, 'new_deaths':0,'cumulative_deaths':0})
    df_slices[j]=pd.concat([pd.DataFrame(start_data),df_slices[j]], ignore_index=True)

df_slices[100]

,date,country_region_code,new_cases,cumulative_cases,new_deaths,cumulative_deaths
0,2020-01-01,IRL,0,0,0,0
1,2020-01-02,IRL,0,0,0,0
2,2020-01-03,IRL,0,0,0,0
3,2020-01-04,IRL,0,0,0,0
4,2020-01-05,IRL,0,0,0,0
...,...,...,...,...,...,...
394,2021-01-29,IRL,2463,193645,47,3167
395,2021-01-30,IRL,247,193892,47,3214
396,2021-01-31,IRL,1411,195303,78,3292
397,2021-02-01,IRL,1244,196547,15,3307


In [94]:
df_cases=pd.concat(df_slices)
df_cases.index=pd.RangeIndex(len(df_cases.index))
df_cases

,country_region_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,ARE,2020-01-01,0.0,0.0,0.0,0.000000,0.000000,0.0
1,ARE,2020-01-02,0.0,0.0,0.0,0.000000,0.000000,0.0
2,ARE,2020-01-03,0.0,0.0,0.0,0.000000,0.000000,0.0
3,ARE,2020-01-04,0.0,0.0,0.0,0.000000,0.000000,0.0
4,ARE,2020-01-05,0.0,0.0,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...
51970,ZWE,2021-01-15,-49.2,-27.0,-46.8,-53.428571,-40.173913,23.4
51971,ZWE,2021-01-16,-47.6,-20.8,-42.8,-56.714286,-34.117647,16.0
51972,ZWE,2021-01-17,-46.2,-26.5,-39.5,-59.400000,-30.333333,18.0
51973,ZWE,2021-01-18,-43.6,-28.2,-41.8,-57.142857,-39.869565,21.8


# Pepare intersection of datasets

In [95]:
df_apple=pd.read_csv("GeospacialDBs_Data/AppleMobilityData_reshaped.csv",header=0)
df_google=pd.read_csv("GeospacialDBs_Data/Global_Mobility_Report_reshaped.csv",header=0)
df_flights=pd.read_csv('GeospacialDBs_Data/Flights_io_per_day_and_country.csv',header=0)

In [96]:
cases_codes=df_cases['country_region_code'].unique()
apple_codes=df_apple['country_region_code'].unique()
google_codes=df_google['country_region_code'].unique()
flights_codes=df_flights['country_region_code'].unique()

In [97]:
intersec_codes=list(set(apple_codes) & set(google_codes) & set(flights_codes) & set(cases_codes))
len(intersec_codes)

60

In [98]:
flights_indices=[]
for i in range(len(df_flights)):
    if (df_flights.at[i,'country_region_code'] not in intersec_codes) or not (df_flights.at[i,'date'].startswith('2020')):
        flights_indices.append(i)

df_flights.drop(flights_indices,inplace=True)
df_flights

,country_region_code,date,origin_count,destination_count
5521,ARE,2020-12-01,8.0,15.0
5522,ARE,2020-12-02,8.0,10.0
5523,ARE,2020-12-03,9.0,12.0
5524,ARE,2020-12-04,13.0,13.0
5525,ARE,2020-12-05,10.0,13.0
...,...,...,...,...
62966,ZAF,2020-08-27,7.0,8.0
62967,ZAF,2020-08-28,5.0,5.0
62968,ZAF,2020-08-29,7.0,9.0
62969,ZAF,2020-08-30,10.0,11.0


In [99]:
df_flights.index=pd.RangeIndex(len(df_flights.index))
df_flights

,country_region_code,date,origin_count,destination_count
0,ARE,2020-12-01,8.0,15.0
1,ARE,2020-12-02,8.0,10.0
2,ARE,2020-12-03,9.0,12.0
3,ARE,2020-12-04,13.0,13.0
4,ARE,2020-12-05,10.0,13.0
...,...,...,...,...
20508,ZAF,2020-08-27,7.0,8.0
20509,ZAF,2020-08-28,5.0,5.0
20510,ZAF,2020-08-29,7.0,9.0
20511,ZAF,2020-08-30,10.0,11.0


In [100]:
count_dict={}
for i in range(len(df_flights)):
    tmp_code=df_flights.at[i,'country_region_code']
    if tmp_code in count_dict:
        count_dict[tmp_code]+=1
    else:
        count_dict[tmp_code]=1

del_countries=[]
for key in count_dict:
    if count_dict[key]!=366:
        del_countries.append(key)
        
for d_key in del_countries:
    del count_dict[d_key]
        
count_dict.values()

dict_values([366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366, 366])

In [101]:
flights_codes=list(count_dict.keys())

In [102]:
intersec_codes=list(set(intersec_codes) & set(flights_codes))
len(intersec_codes)

48

In [87]:
flights_indices=[]
for i in range(len(df_flights)):
    if (df_flights.at[i,'country_region_code'] not in intersec_codes) or not (df_flights.at[i,'date'].startswith('2020')):
        flights_indices.append(i)

df_flights.drop(flights_indices,inplace=True)
df_flights

,country_region_code,date,origin_count,destination_count
0,ARE,2020-12-01,8.0,15.0
1,ARE,2020-12-02,8.0,10.0
2,ARE,2020-12-03,9.0,12.0
3,ARE,2020-12-04,13.0,13.0
4,ARE,2020-12-05,10.0,13.0
...,...,...,...,...
16831,ZAF,2020-08-27,7.0,8.0
16832,ZAF,2020-08-28,5.0,5.0
16833,ZAF,2020-08-29,7.0,9.0
16834,ZAF,2020-08-30,10.0,11.0


In [88]:
cases_indices=[]
for i in range(len(df_cases)):
    if (df_cases.at[i,'country_region_code'] not in intersec_codes) or not (df_cases.at[i,'date'].startswith('2020')):
        cases_indices.append(i)

df_cases.drop(cases_indices,inplace=True)
df_cases

KeyError: 0

In [31]:
apple_indices=[]
for i in range(len(df_apple)):
    if (df_apple.at[i,'country_region_code'] not in intersec_codes) or not (df_apple.at[i,'date'].startswith('2020')):
        apple_indices.append(i)

df_apple.drop(apple_indices,inplace=True)
df_apple            

,date,country_region_code,driving_percent_change_from_baseline,walking_percent_change_from_baseline
732,2020-01-01,AUS,0.000000,0.000000
733,2020-01-02,AUS,-8.439936,31.683246
734,2020-01-03,AUS,-14.787957,54.279986
735,2020-01-04,AUS,-19.044063,67.790219
736,2020-01-05,AUS,-21.208255,72.213945
...,...,...,...,...
22321,2020-12-27,USA,-11.140000,-17.000000
22322,2020-12-28,USA,6.730000,0.750000
22323,2020-12-29,USA,9.790000,5.190000
22324,2020-12-30,USA,14.740000,11.110000


In [ ]:
df_flights.to_csv(r'GeospacialDBs_Data/Flights_io_per_day_and_country.csv', index = False)
df_cases.to_csv(r'GeospacialDBs_Data/WHO-COVID-19-global-data_reshaped.csv', index = False)
df_apple.to_csv(r"GeospacialDBs_Data/AppleMobilityData_reshaped.csv", index = False)